In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [ ]:
# Constants
CONNECTOR_NAME = "binance_perpetual"
INTERVAL = "1m"
DAYS = 20

# Features configuration
VOLATILITY_WINDOW = 100
BOLLINGER_LENGTH = 200
BOLLINGER_STD = 2.0

# Download data
- Get trading rules
- Get candles for the last x days

In [ ]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset("USDT")\
    .filter_by_min_notional_size(Decimal("5"))\
    .get_all_trading_pairs()

In [ ]:
tasks = [clob.get_candles_last_days(
    connector_name=CONNECTOR_NAME,
    trading_pair=trading_pair,
    interval=INTERVAL,
    days=DAYS,
) for trading_pair in trading_pairs[:5]]
candles = await asyncio.gather(*tasks)
candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}

In [ ]:
clob.dump_candles_cache(os.path.join(root_path, "data"))

In [ ]:
clob.candles_cache.keys()

In [ ]:
os.listdir(os.path.join(os.path.join(root_path, "data"), "candles"))

In [ ]:
clob.load_candles_cache(os.path.join(root_path, "data"))

In [ ]:
candles.keys()

In [ ]:
trading_pair = list(candles.keys())[0]
candles[trading_pair].plot(type="candles")

In [ ]:
candles[trading_pair].plot(type="returns")

In [ ]:
import pandas_ta as ta

def apply_signal(x):
    if x >= 1:
        return -1
    elif x <= 0:
        return 1
    else:
        return 0


In [ ]:
import pandas as pd

# Assuming df has already been created and processed as per your initial steps
df = candles[trading_pair].data
df.ta.bbands(length=BOLLINGER_LENGTH, std=BOLLINGER_STD, append=True)
df["signal"] = df[f"BBP_{BOLLINGER_LENGTH}_{BOLLINGER_STD}"].apply(apply_signal)
df["out_of_bounds"] = df["signal"].diff().fillna(0)
df.loc[df["signal"] == 0, "out_of_bounds"] = 0

# Create df_filtered for values where out_of_bounds is not 0
df_filtered = df[df["out_of_bounds"] != 0].copy()
df["score"] = 0

score = (df.loc[df["out_of_bounds"] != 0, "out_of_bounds"].shift() != df.loc[df["out_of_bounds"] != 0, "out_of_bounds"]).cumsum()

n_out_of_bounds = len(df[df["out_of_bounds"] != 0])

df.loc[df_filtered.index, "score"] = score
df_filtered = df[df["score"] != 0].copy()
df_filtered["changes"] = df_filtered["score"].shift() != df_filtered["score"]
timestamps_change = df_filtered[df_filtered["changes"] != 0].index

prices = []

for i, timestamp in enumerate(timestamps_change):
    if i == len(timestamps_change) - 1:
        break
    next_timestamp = timestamps_change[i + 1]
    df_temp = df[timestamp:next_timestamp].copy()
    max_price = df_temp["high"].max()
    min_price = df_temp["low"].min()
    start_price = df_temp["close"].iloc[0]
    end_price = df_temp["close"].iloc[-1]
    signal = df_temp["signal"].iloc[0]
    if signal == 1:
        reversion_price = min(end_price, max_price)
        worst_deviation = abs(min_price - start_price) / start_price
        total_reversion = (reversion_price - start_price) / start_price
    elif signal == -1:
        reversion_price = max(end_price, min_price)
        worst_deviation = abs(max_price - start_price)  / start_price
        total_reversion = -1 * (reversion_price - start_price) / start_price
    else:
        worst_deviation = 0
        total_reversion = 0

    prices.append({
        "start_timestamp": timestamp,
        "end_timestamp": next_timestamp,
        "start_price": start_price,
        "max_price": max_price,
        "min_price": min_price,
        "end_price": end_price,
        "signal": signal,
        "worst_deviation": worst_deviation,
        "total_reversion": total_reversion,
    })
prices

In [ ]:
import plotly.graph_objects as go


candles_figure = candles[trading_pair].fig()
candles_figure.add_trace(go.Scatter(x=df.index,
                                    y=df[f"BBU_{BOLLINGER_LENGTH}_{BOLLINGER_STD}"])
                        )
candles_figure.add_trace(go.Scatter(x=df.index,
                                    y=df[f"BBM_{BOLLINGER_LENGTH}_{BOLLINGER_STD}"])
                        )
candles_figure.add_trace(go.Scatter(x=df.index,
                                    y=df[f"BBL_{BOLLINGER_LENGTH}_{BOLLINGER_STD}"])
                        )
for price in prices:
    # Add line from start to end price
    candles_figure.add_trace(go.Scatter(
        x=[price['start_timestamp'], price['end_timestamp']],
        y=[price['start_price'], price['end_price']],
        mode='lines+markers',
        name=f"Signal {price['signal']}",
        line=dict(color='blue' if price['signal'] == 1 else 'red')
    ))
candles_figure.show()

In [ ]:
worst_deviations = [price["worst_deviation"] for price in prices]
fig = go.Figure(go.Histogram(x=worst_deviations))
fig.show()

In [ ]:
total_reversion = [price["total_reversion"] for price in prices]
fig = go.Figure(go.Histogram(x=total_reversion))
fig.show()

In [ ]:
pd.Series(total_reversion).describe()

In [ ]:
fake_reversions = [r for r in total_reversion if r < 0]
right_reversions = [r for r in total_reversion if r > 0]

# Metrics
worst_q0 = pd.Series(worst_deviations).quantile(0)
worst_q1 = pd.Series(worst_deviations).quantile(0.25)
worst_q2 = pd.Series(worst_deviations).quantile(0.5)
worst_q3 = pd.Series(worst_deviations).quantile(0.75)
worst_q4 = pd.Series(worst_deviations).quantile(1)

right_rev_q0 = pd.Series(right_reversions).quantile(0)
right_rev_q1 = pd.Series(right_reversions).quantile(0.25)
right_rev_q2 = pd.Series(right_reversions).quantile(0.5)
right_rev_q3 = pd.Series(right_reversions).quantile(0.75)
right_rev_q4 = pd.Series(right_reversions).quantile(1)

n_right_reversions = len(right_reversions)
n_fake_reversions = len(fake_reversions)
street_cross = len(prices)


print(f"Fake reversions: {n_fake_reversions/len(total_reversion)} | Right reversions: {n_right_reversions/len(total_reversion)}")

In [ ]:
fig = go.Figure(go.Box(x=[r for r in total_reversion if r > 0]))
fig.show()

In [ ]:
fig = go.Figure(go.Box(x=worst_deviations))
fig.show()

In [ ]:
n_out_of_bounds

In [ ]:
df["timestamp"].min()

In [ ]:
df["timestamp"].max()

In [ ]:
candles_figure = candles["NEAR-USDT"].plot(type="candles")

In [ ]:
from features.candles.trend import TrendConfig
from features.candles.volatility import VolatilityConfig
from features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    trend_config=TrendConfig(short_window=TREND_SHORT_WINDOW, long_window=TREND_LONG_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

In [ ]:
from research_notebooks.dneitor.utils import filter_top_markets

TOP_X_MARKETS = 10  # Number of top markets to select
# Assuming the `top_markets` DataFrame has been generated as shown in the previous steps
# Constants
TOTAL_AMOUNT = 3000  # General total amount for all markets
MIN_AMOUNT_PER_MARKET = 200  # Minimum amount per market
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market

# Example values
TAKE_PROFIT_MULTIPLIER = 0.3  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster
top_markets = filter_top_markets(report_df=report, top_x=TOP_X_MARKETS, min_volume_usd=2000, min_atr=0.008, trend_threshold=-0.5)
top_markets

In [ ]:
from research_notebooks.dneitor.utils import generate_config, dump_dict_to_yaml

strategy_config = generate_config(
    id="dneitor-binance_0.4",
    connector_name=CONNECTOR_NAME,
    candles=candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_amount_per_market=MIN_AMOUNT_PER_MARKET,
)
dump_dict_to_yaml("configs/", strategy_config)